## 3.Data Preparation

On the data preparation stage the objective is prepare the data for the modeling stage. To accoplish this objective, we will utilise the tasks raised by the preceding stage, the data analysis stage. At the end of the third stage we will have the data tables to proceed with the modeling. 

### A)Selecting the data

Let's start loading the raw data given by the Kaggle competition website.

In [75]:
import pandas as pd
#loading the data
gender_submission = pd.read_csv("../data/gender_submission.csv")
test_ds = pd.read_csv("../data/test.csv")
train_ds = pd.read_csv("../data/train.csv")

In [76]:
train_ds.iloc[872]

PassengerId                         873
Survived                              0
Pclass                                1
Name           Carlsson, Mr. Frans Olof
Sex                                male
Age                                33.0
SibSp                                 0
Parch                                 0
Ticket                              695
Fare                                5.0
Cabin                       B51 B53 B55
Embarked                              S
Name: 872, dtype: object

In [77]:
train_ds.head(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


Before dropping the cabin, lets divide the fare by the number of cabins. Therefore, we will have a more precise fare by ticket.

In [78]:
type(train_ds['Cabin'][1]) is (type(0.1))

False

Just counting the letters will do the job.

In [79]:
def count_space(string):
    c=1
    for i in string:
        if(i.isspace()):
            c+=1
    return c
def div_fare_cabin(ds):
    new_fare = []
    for _,row in ds.iterrows():
        if type(row['Cabin']) is not type(0.1):
            new_fare.append(row['Fare']/count_space(row['Cabin']))
        else:
            new_fare.append(row['Fare'])
    return new_fare
train_ds['Fare'] = div_fare_cabin(train_ds)
test_ds['Fare'] = div_fare_cabin(test_ds)

the analysis pointed useless columns: PassagerId, Name. Cabin and ticket, maybe is not useless but has missing values and alphabetic caracters and will be droped to.

In [80]:
columns_droped = ["PassengerId", "Ticket", "Cabin", "Name"]
train_ds.drop(labels=columns_droped, axis=1, inplace=True)
train_ds.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


Making the same changes on the test dataset.

In [81]:
test_ds.drop(labels=columns_droped, axis=1, inplace=True)
test_ds.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S


### B)Missing Values

The cabin, age, embarked are the three columns with missing values. The column cabin is already removed by the last procedure. Next we opted for changing the missing value with the mean of the column. This is a simple stategy to handle the missing values. The reason behind selecting this strategy is two: there is no easy way to determine the age with the others columns, and the data available is to small to drop the missing. There is more than 10% of available data with missing age, droping the rows is not a valid option. 

In [82]:
train_ds['Age'].mean()

29.69911764705882

Let's put change the missing values to 30. For the scalers, the test set cannot have a value below the minimun of the training set, so we will use the minimum and the max of the training set as a threshold.

In [83]:
test_ds[test_ds['Age'] < min(train_ds['Age'])]

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
201,3,male,0.33,0,2,14.400,S
354,3,female,0.17,1,2,20.575,S


In [84]:
new_age = []
for _,row in test_ds.iterrows():
    if row['Age']<0.42:
        new_age.append(0.50)
    else:
        new_age.append(row['Age'])
test_ds['Age'] = new_age

In [85]:
train_ds['Age'].fillna(value=30, inplace=True)
test_ds['Age'].fillna(value=30, inplace=True)

To impute the fare missing value on the test, we gonna use a more sofisticated imputation. Intuitively the pclass is a good guess for the fare, so we will put the mean considering the pclass.

In [86]:
test_ds[test_ds['Fare'].isna()]

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
152,3,male,60.5,0,0,NaN,S


In [87]:
train_ds[train_ds['Pclass'] == 3]['Fare'].mean()

13.629411303462323

13.63 is a better guess for the missing value on the test set. If we had opted on the overall mean, the imputation value would be 32, a much greater number.

In [88]:
test_ds['Fare'].fillna(value=13.63, inplace=True)

There is two values missing in the embarked column for the training set, we will just drop them.

In [89]:
train_ds.drop(train_ds[train_ds['Embarked'].isnull()].index, inplace=True)

The test doesn't has any missing value for the embarked column.

### C)Transform data

In [90]:
#selecting the numeric features.
numeric_features = ['Age', 'Fare']

In [91]:
train_ds[(train_ds['Fare']<5) & (train_ds['Fare']>0)]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
75,0,3,male,25.0,0,0,3.825000,S
378,0,3,male,20.0,0,0,4.012500,C
699,0,3,male,42.0,0,0,3.825000,S
715,0,3,male,19.0,0,0,3.825000,S
872,0,1,male,33.0,0,0,1.666667,S


In [92]:
train_ds['Fare'].replace(to_replace=0, value=4.0125, inplace = True)
test_ds['Fare'].replace(to_replace=0, value=4.0125, inplace = True)

In [93]:
train_ds.loc[(train_ds['Fare']<5) & (train_ds['Fare']>0), 'Pclass'] = 3

In [94]:
test_ds.loc[(test_ds['Fare']<5) & (test_ds['Fare']>0), 'Pclass'] = 3

Setting an upper limit in Parch and Sibsp. Substitutions help the model see patterns on the data.

In [95]:
new_sibsp = []
new_parch = []
def set_suplimit(ds, limit, row_name):
    new_feat = []
    for _,row in ds.iterrows():
        if row[row_name] >= limit:
            new_feat.append(limit)
        else:
            new_feat.append(row[row_name])
    return new_feat

#Train set
train_ds['Parch'] = set_suplimit(train_ds, 3, 'Parch')
train_ds['SibSp'] = set_suplimit(train_ds, 4, 'SibSp')

#Test set
test_ds['Parch'] = set_suplimit(test_ds, 3, 'Parch')
test_ds['SibSp'] = set_suplimit(test_ds, 4, 'SibSp')

Lets set an upper limit for the fare and age.

Checking the age on the exploratory analysis, stage 2, we can see that sixty years old dosn't has much diferrence than the seventy five. So lets put sixty as a upper limit.

In [96]:
train_ds.loc[train_ds['Age']>60, 'Age'] = 60
train_ds.loc[train_ds['Age']== 60]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
33,0,2,male,60.0,0,0,10.500000,S
54,0,1,male,60.0,0,1,61.979200,C
96,0,1,male,60.0,0,0,34.654200,C
116,0,3,male,60.0,0,0,7.750000,Q
170,0,1,male,60.0,0,0,33.500000,S
252,0,1,male,60.0,0,0,26.550000,S
275,1,1,female,60.0,1,0,77.958300,S
280,0,3,male,60.0,0,0,7.750000,Q
326,0,3,male,60.0,0,0,6.237500,S
366,1,1,female,60.0,1,0,75.250000,C


In [97]:
test_ds.loc[test_ds['Age']>60, 'Age'] = 60
test_ds.loc[test_ds['Age']== 60]

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
2,2,male,60.0,0,0,9.687500,Q
13,2,male,60.0,1,0,26.000000,S
48,1,female,60.0,0,0,76.291700,C
69,1,female,60.0,1,3,87.666667,S
81,1,male,60.0,1,0,110.889600,S
96,1,female,60.0,1,0,78.850000,S
114,1,female,60.0,1,0,110.889600,S
142,1,male,60.0,1,3,65.593750,C
152,3,male,60.0,0,0,13.630000,S
179,1,female,60.0,0,2,83.158300,C


Now for the fare.

In [98]:
#train_ds.loc[train_ds['Age']>60, 'Age'] = 60
train_ds.loc[train_ds['Fare']> 100]


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
31,1,1,female,30.0,1,0,146.5208,C
118,0,1,male,24.0,0,1,123.7604,C
195,1,1,female,58.0,0,0,146.5208,C
215,1,1,female,31.0,1,0,113.2750,C
258,1,1,female,35.0,0,0,512.3292,C
268,1,1,female,58.0,0,1,153.4625,S
269,1,1,female,35.0,0,0,135.6333,S
299,1,1,female,50.0,0,1,123.7604,C
306,1,1,female,30.0,0,0,110.8833,C
307,1,1,female,17.0,1,0,108.9000,C


In [99]:
train_ds.loc[train_ds['Fare']> 100, 'Fare'] = 100
test_ds.loc[test_ds['Fare']> 100, 'Fare'] = 100

In [100]:
train_ds.loc[train_ds['Fare']== 100]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
31,1,1,female,30.0,1,0,100.0,C
118,0,1,male,24.0,0,1,100.0,C
195,1,1,female,58.0,0,0,100.0,C
215,1,1,female,31.0,1,0,100.0,C
258,1,1,female,35.0,0,0,100.0,C
268,1,1,female,58.0,0,1,100.0,S
269,1,1,female,35.0,0,0,100.0,S
299,1,1,female,50.0,0,1,100.0,C
306,1,1,female,30.0,0,0,100.0,C
307,1,1,female,17.0,1,0,100.0,C


In [101]:
test_ds.loc[test_ds['Fare'] == 100]

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
59,1,female,36.0,0,0,100.0,C
74,1,female,35.0,0,0,100.0,C
75,1,male,32.5,0,0,100.0,C
81,1,male,60.0,1,0,100.0,S
114,1,female,60.0,1,0,100.0,S
141,1,female,33.0,0,0,100.0,S
156,1,female,29.0,0,0,100.0,S
184,1,female,27.0,1,1,100.0,C
196,1,male,6.0,0,2,100.0,C
202,1,male,47.0,1,0,100.0,C


Combine features can explain some relations on the data.

Some columns has categorical values which needs to be transformed. We will use one hot enconding.

In [103]:
from sklearn.preprocessing import OneHotEncoder
cat_columns = ['Pclass', 'Sex', 'Embarked', 'SibSp', 'Parch']
one = OneHotEncoder(sparse_output=False).fit(X=train_ds[cat_columns])

In [104]:
one.get_feature_names_out()

array(['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male',
       'Embarked_C', 'Embarked_Q', 'Embarked_S', 'SibSp_0', 'SibSp_1',
       'SibSp_2', 'SibSp_3', 'SibSp_4', 'Parch_0', 'Parch_1', 'Parch_2',
       'Parch_3'], dtype=object)

In [105]:
test_ds[cat_columns].describe()

,Pclass,SibSp,Parch
count,418.000000,418.000000,418.000000
mean,2.275120,0.425837,0.346890
std,0.838752,0.755794,0.717545
min,1.000000,0.000000,0.000000
25%,1.250000,0.000000,0.000000
50%,3.000000,0.000000,0.000000
75%,3.000000,1.000000,0.000000
max,3.000000,4.000000,3.000000


In [106]:
#Transforming and adding the new columns on the data set
train_ds[one.get_feature_names_out()] = one.transform(train_ds[cat_columns])
test_ds[one.get_feature_names_out()] = one.transform(test_ds[cat_columns])

Verifying the transformation.

In [107]:
train_ds.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Pclass_1,Pclass_2,...,Embarked_S,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,Parch_0,Parch_1,Parch_2,Parch_3
0,0,3,male,22.0,1,0,7.2500,S,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,1,female,38.0,1,0,71.2833,C,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,3,female,26.0,0,0,7.9250,S,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1,1,female,35.0,1,0,53.1000,S,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0,3,male,35.0,0,0,8.0500,S,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


Now we can drop the original columns.

In [108]:
train_ds.drop(one.feature_names_in_, axis=1, inplace=True)
test_ds.drop(one.feature_names_in_, axis=1, inplace=True)

In [109]:
train_ds.head()

,Survived,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,Parch_0,Parch_1,Parch_2,Parch_3
0,0,22.0,7.2500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,38.0,71.2833,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,26.0,7.9250,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1,35.0,53.1000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0,35.0,8.0500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


The last transformation is apply some scaler on the numeric features. On the scenario presented by the data analysis a simple MinMaxScaler will work good. We can see that without the standardization, the Fare probaly will have a greater influence on the model because of the bigger numbers. Applying the standardization, we eliminate the influence by bigger numbers. We can see this as a 'fare play' between the features, every feature is treated equal by the model.

In [110]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler



std_scaler = MinMaxScaler((-1,1)).fit(X=train_ds[numeric_features])


In [111]:
test_ds.iloc[201]

Age            0.5
Fare          14.4
Pclass_1       0.0
Pclass_2       0.0
Pclass_3       1.0
Sex_female     0.0
Sex_male       1.0
Embarked_C     0.0
Embarked_Q     0.0
Embarked_S     1.0
SibSp_0        1.0
SibSp_1        0.0
SibSp_2        0.0
SibSp_3        0.0
SibSp_4        0.0
Parch_0        0.0
Parch_1        0.0
Parch_2        1.0
Parch_3        0.0
Name: 201, dtype: float64

In [112]:
train_ds[numeric_features] = std_scaler.transform(train_ds[numeric_features])
test_ds[numeric_features] = std_scaler.transform(test_ds[numeric_features])

#train_ds[fare_features] = mm_scaler.transform(train_ds[fare_features])
#test_ds[fare_features] = mm_scaler.transform(test_ds[fare_features])

In [113]:
test_ds.iloc[201]

Age          -0.997315
Fare         -0.741017
Pclass_1      0.000000
Pclass_2      0.000000
Pclass_3      1.000000
Sex_female    0.000000
Sex_male      1.000000
Embarked_C    0.000000
Embarked_Q    0.000000
Embarked_S    1.000000
SibSp_0       1.000000
SibSp_1       0.000000
SibSp_2       0.000000
SibSp_3       0.000000
SibSp_4       0.000000
Parch_0       0.000000
Parch_1       0.000000
Parch_2       1.000000
Parch_3       0.000000
Name: 201, dtype: float64

In [114]:
train_ds.describe()

,Survived,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,Parch_0,Parch_1,Parch_2,Parch_3
count,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000
mean,0.382452,-0.021599,-0.495492,0.233971,0.200225,0.565804,0.350956,0.649044,0.188976,0.086614,0.724409,0.681665,0.235096,0.031496,0.017998,0.033746,0.760405,0.132733,0.089989,0.016873
std,0.486260,0.421658,0.533596,0.423592,0.400394,0.495930,0.477538,0.477538,0.391710,0.281427,0.447063,0.466093,0.424297,0.174752,0.133018,0.180676,0.427077,0.339477,0.286327,0.128868
min,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,-0.275596,-0.873306,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,0.000000,-0.007049,-0.739915,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,1.000000,0.160792,-0.403390,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Before ending this section, we will save the scaler and the encoder.

In [115]:
import pickle

with open("../var/hotencoder", "wb") as f:
    pickle.dump(one, f)
with open("../var/scaler", "wb") as f:
    pickle.dump(std_scaler,f)

### D)Spliting the data

Now we will split the data in y(targets) and X(features), respectively for the train and test data set.

In [116]:
y_train = train_ds['Survived']
x_train = train_ds.drop('Survived', axis=1)

#Test doesn't have the target in Kaggle competition
x_test = test_ds

Saving the data.

In [117]:
data_path = "../data"
y_train.to_csv(f"{data_path}/y_train.csv")
x_train.to_csv(f"{data_path}/x_train.csv")
x_test.to_csv(f"{data_path}/x_test.csv")